# Mitigating Polarisation in Online Discussions Through Adaptive Moderation Techniques

## Model selection

We choose quantized versions of the LLaMa-13b-chat variant. Previous experiments which used the LLaMa-13b base model yielded unsatisfactory results. The models follow the GGUF format which is used by the `llama.cpp` project, on which the high-level Python library is based on.

The quantization method was selected to be highly accurate while keeping inference relatively fast. We don't care about model size since the model is lazily loaded from the file cache due to Linux file-cached memory files (see comments below). *If you intend to run this notebook on Windows or MacOS make sure the RAM can hold the whole model at once*.

Model selection and download was performed using the [following HuggingFace repository](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF).

We use the `llama-ccp-python` library to run the model locally (not to be confused with the `pyllama-cpp` library).

In [ ]:
# download only if not exists
![ ! -f ../models/llama-2-13b-chat.Q5_K_M.gguf ] && wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf?download=true -O ../models/llama-2-13b-chat.Q5_K_M.gguf

In [ ]:
import sys 
import os


sys.path.append(os.path.abspath('..'))

In [ ]:
%load_ext autoreload
%autoreload 2

import llama_cpp

import tasks.models
from tasks.actors import IActor, LLMUser, LLMAnnotator
import tasks.conversation
import tasks.util


OUTPUT_DIR = "output"

MAX_TOKENS = 512
# see this for a discussion on ctx width for llama models
# https://github.com/ggerganov/llama.cpp/issues/194
CTX_WIDTH_TOKENS = 1024 
MODEL_PATH = "../models/llama-2-13b-chat.Q5_K_M.gguf"
RANDOM_SEED = 42
INFERENCE_THREADS = 4


llm = llama_cpp.Llama(
      model_path=MODEL_PATH,
      seed=RANDOM_SEED,
      n_ctx=CTX_WIDTH_TOKENS,
      n_threads=INFERENCE_THREADS,
      # will vary from machine to machine
      n_gpu_layers=12,
      # if ran on Linux, model size does not matter since the model uses mmap for lazy loading
      # https://github.com/ggerganov/llama.cpp/discussions/638
      # still have to pay some performance costs of course
      use_mmap=True,
      # using llama-2 as chat format leads to well-known model collapse
      # https://www.reddit.com/r/LocalLLaMA/comments/17th1sk/cant_make_the_llamacpppython_respond_normally/
      chat_format="alpaca", 
      mlock=True, # keep memcached model files in RAM if possible
      verbose=False,
)

When using `create_completion()` instead of `create_chat_completion()`, the model refuses to answer at all when the prompt becomes larger than a few sentences. (https://github.com/run-llama/llama_index/issues/8973).

The model is also extremely sensitive to the prompt template, frequently producing no output (https://huggingface.co/TheBloke/Nous-Capybara-34B-GGUF/discussions/4)

In [ ]:
llm("Q: You are an assistant who specializes in computer science. Describe what Linux is A: ",
              max_tokens=32, 
              stop=["Q:", "\n"], 
              echo=True)

In [ ]:
llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are an assistant who specializes in computer science.",
        },
        {"role": "user", "content": "Describe what Linux is."},
    ],
    max_tokens=MAX_TOKENS,
    # prevent model from making up its own prompts
    # may need tuning depending on llm chat_format parameter
    stop=["###", "\n"],
)

## Setting up a conversation

We create our own playground, in which models pretending to be users take turns participating in the discussion. In part based on [Bootstrapping LLM-based Task-Oriented Dialogue Agents via Self-Talk](https://arxiv.org/abs/2401.05033), with the difference being that instead of a client and an agent, we have two clients and an agent interacting with each other and with no specific goal in mind.

Our playground consists of three parts: *Models*, *Actors* and the *Conversation*.
* **Models** are wrappers around actual LLMs in order to freely tweak LLM behavior without compromising the rest of our API
* **Actors** are objects that define a prompt template and apply it to Models.
    * Actors could also be *human*, *IR-based models* or just *sophisticated random samplers* as seen in [DeliData: A dataset for deliberation in multi-party problem solving](https://arxiv.org/abs/2108.05271)
* The conversation is handled by the **ConversationManager** which gives each Actor a turn to speak and records the history of the dialogue. It's also responsible for determining which parts of the coversation history are fed as context to each Actor.


In [ ]:
model = tasks.models.LlamaModel(llm, max_out_tokens=MAX_TOKENS, seed=RANDOM_SEED)

In [ ]:
userA_name = "Steve2001"
userB_name = "GeorgeBush78"
userA: IActor = LLMUser(
    model=model,
    name=userA_name,
    role="chat user",
    attributes=["suspicious", "open-minded", "level-headed"],
    context=f"Argue with {userB_name}. Claim that abortion is ethical and should be freely provided when necessary. Respond in kind if sufficiently provoked. Remember that you do not want to get banned.",
    instructions="",
)

userB: IActor = LLMUser(
    model=model,
    name=userB_name,
    role="chat user",
    attributes=["compomising", "passionate", "frustrated"],
    context=f"Argue with {userA_name}. Claim that abortion is inherently immoral. You may be rude, aggresive and use personal insults where appropriate. Remember that you do not want to get banned.",
    instructions=f"Disagree with {userA_name}.",
)

moderator: IActor = LLMUser(
    model=model,
    name="moderator01",
    role="chat moderator",
    attributes=["just", "cautious", "strict"],
    context="Moderate a discussion about abortion.",
    instructions="Intervene if one user dominates or veers off-topic. Respond only if necessary. Write '<No response>' if intervention is unecessary. Be firm and threaten to displine non-cooperating users.",
)

## Setting a conversation manually

### With a moderator

In [ ]:
chat = tasks.conversation.Conversation(
    users=[userA, userB], moderator=moderator, history_context_len=3, conv_len=3
)
chat.begin_conversation(verbose=True)

Every Conversation instance can be converted to a dictionary form in order to programmatically view and manipulate its contents:

In [ ]:
chat.to_dict()

A conversation can be serialized in JSON form with an automatic naming scheme. The file contains all necessary metadata as well as the messages themselves.

Uncomment the block below to see how it works.

In [ ]:
#output_path = tasks.util.generate_datetime_filename(output_dir=OUTPUT_DIR)
#chat.to_json_file(output_path)

### Without a moderator

In [ ]:
chat = tasks.conversation.Conversation(
    users=[userA, userB], history_context_len=3, conv_len=3
)
chat.begin_conversation(verbose=True)

## Creating a conversation from serialized input data

In [ ]:
data = tasks.conversation.LLMConvData.from_json_file("../data/polarized_1.json")
generator = tasks.conversation.LLMConvGenerator(data=data, user_model=model, moderator_model=model)
conv = generator.produce_conversation()

print("Beginning conversation...")
conv.begin_conversation(verbose=True)
print(conv)

## Using the python script

Note that using the scripts from within the notebook will load the model twice, probably leading to a crash if you are using CUDA. Thus, their cells are set to "Raw".

## Automating the creation of synthetic dialogues

## Annotating the synthetic dialogues via LLMs

Since annotation can be modelled as a discussion between the input data and the annotator, we can use a similar API with above in order to facilitate it.

The library components for annotation can be found in `tasks.annotation`.

### Using the programming API

In [ ]:
import tasks.annotation


annotator: IActor = LLMAnnotator(
    model=model,
    name="",
    role="annotator",
    attributes=["expert", "neutral"],
    context=""
    instructions="You, the annotator, will be given a series of comments made on an online discussion. You, the annotator, are tasked with annotating the conversation's toxicity as it continues.\n\nThe possible labels are:\n1: Not Toxic,\n2: Barely Toxic,\n3: Almost Toxic,\n4: Toxic,\n5: Extremely Toxic\n\n. Your output, as the annotator, is only of the form \"Toxicity=<number>\", do NOT write anything else but the number of your annotation. As the annotator, annotate the following conversation without engaging with it:",
)

conv = tasks.annotation.AnnotationConv(annotator=annotator, 
                                        conv_logs_path="../output/conversations/game_improved/24-07-19-13-36.json", history_ctx_len=4 )
conv.begin_annotation()
conv.to_json_file("../output/annotation/demo/programming_api_example.json")

### Using the JSON-based API

In [ ]:
model = tasks.models.LlamaModel(llm, max_out_tokens=MAX_TOKENS, seed=RANDOM_SEED)
data = tasks.annotation.LLMAnnotatorData.from_json_file("../data/annotations/annot_neutral.json")
gen = tasks.annotation.LLMAnnotationGenerator(data, model, conv_logs_path="../output/conversations/game_improved/24-07-19-13-36.json")
conv = gen.produce_conversation()

conv.begin_annotation(verbose=True)
output_path = tasks.util.generate_datetime_filename(output_dir="../output/annotation/demo", file_ending=".json")
conv.to_json_file(output_path)

### Using the automatic script